<a href="https://colab.research.google.com/github/Charan6924/ShakespeareGPT/blob/main/GPTDev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-05 23:45:23--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.007s  

2026-01-05 23:45:23 (145 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# Imports
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
torch.manual_seed(1337)

In [3]:
n_embed = 384
block_size = 256
dropout = 0.2

class Head(nn.Module):
    def __init__(self,head_size):
      super().__init__()
      self.key = nn.Linear(n_embed,head_size,bias=False)
      self.query = nn.Linear(n_embed,head_size,bias=False)
      self.value = nn.Linear(n_embed,head_size,bias=False)
      self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
      self.dropout = nn.Dropout(dropout)

    def forward(self,x):
      B,T,C = x.shape
      k = self.key(x)
      q = self.query(x)

      wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
      wei = wei.masked_fill(self.tril[:T,:T]==0,float('-inf'))  # type: ignore
      wei = F.softmax(wei,dim=-1)
      wei = self.dropout(wei)
      v = self.value(x)
      out = wei @ v
      return out

# Multiple attentions
class MultiHeadAttention(nn.Module):
    """multiple attention heads in parallel"""
    def __init__(self,num_heads,head_size):
      super().__init__()
      self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
      self.proj = nn.Linear(num_heads * head_size,n_embed)
      self.dropout = nn.Dropout(dropout)

    def forward(self,x):
       out = torch.cat([head(x) for head in self.heads],dim = -1)
       out = self.proj(out)
       return out

In [4]:
# Block
class Block(nn.Module):
    def __init__(self,n_embed,n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head,head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self,x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [5]:
dropout = 0.2

# Feedforward network
class FeedForward(nn.Module):
    def __init__(self,n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed,n_embed),
            nn.ReLU(),
            nn.Linear(n_embed,n_embed), #projection layer
            nn.Dropout(dropout)
        )

    def forward(self,x):
        return self.net(x)

In [8]:
# Hyperparameters
batch_size = 64
block_size = 256
max_iters = 2500
eval_interval = 300
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 384
n_head = 6
n_layer = 6

# Preprocessing
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for i,s in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s]
decode = lambda l : "".join([itos[ix] for ix in l])
data = torch.tensor(encode(text), dtype = torch.long)

# Splits
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# Data Loader
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x = x.to(device)
    y = y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Model
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embed)
    self.position_embedding_table = nn.Embedding(block_size,n_embed)
    self.blocks = nn.Sequential(
        Block(n_embed,n_head),
        Block(n_embed,n_head),
        Block(n_embed,n_head),
        Block(n_embed,n_head),
        Block(n_embed,n_head),
        Block(n_embed,n_head),
        nn.LayerNorm((n_embed)))
    self.lm_head = nn.Linear(n_embed,vocab_size)

  def forward(self,idx,targets=None):
    tok_emb = self.token_embedding_table(idx) # (B,T,C)
    # (B,T,vocab_size)
    pos_emb = self.position_embedding_table(torch.arange(idx.shape[1], device=device)) # (T,C)
    x = tok_emb + pos_emb # (B,T,C)
    x = self.blocks(x)
    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits,loss

  def generate(self,idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:,-block_size:]
      logits,loss = self(idx_cond)
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim = 1)
      idx_next = torch.multinomial(probs, num_samples = 1)
      idx = torch.cat((idx,idx_next),dim=1)
    return idx

# Instantiate model and optimizer
model = BigramLanguageModel()
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

# Training
print(f'evaluating model {device}')
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

evaluating model cuda
step 0: train loss 4.3518, val loss 4.3543
step 300: train loss 1.8056, val loss 1.9323
step 600: train loss 1.5001, val loss 1.6788
step 900: train loss 1.3771, val loss 1.5945
step 1200: train loss 1.3014, val loss 1.5467
step 1500: train loss 1.2444, val loss 1.5176
step 1800: train loss 1.2035, val loss 1.5062
step 2100: train loss 1.1636, val loss 1.4997
step 2400: train loss 1.1312, val loss 1.5053

Where he is possible castle, that
hadst thou not pursuit, or in it in his rock grave;
The wearing shows faults, to cowards are; and serve,
I bent lies, I chase dishonour'd on Rome.

CORIOLANUS:
Marcius doth not the prove is the morth,
For whatsoever sit to-gao
Of He applot of this bed my friends,
Sending om the bloody deep.

First Coriol:
Anon that I advert in Lord Aumerle to taught:
O pray my with weak, I say never entreaties to agre?
One so Bondaggordon that has get to chese,
That might never 
